In [1]:
from google.colab import drive

drive.mount("/content/MyDrive")

Mounted at /content/MyDrive


In [ ]:
import scipy.io
import pandas as pd


In [ ]:


#mat = scipy.io.loadmat('/content/MyDrive/MyDrive/Colab Notebooks/Project/url.mat')
#mat = {k:v for k, v in mat.items() if k[0] != '_'}
#data = pd.DataFrame({k: pd.DataFrame(v[0]) for k, v in mat.items()})

In [2]:
path = "/content/MyDrive/MyDrive/네트워크 과정 프젝/"

In [ ]:
oridf = pd.read_csv(path+'malicious_phish.csv')

In [21]:
# MODEL_process_ver

#!pip -q install transformers
#HuggingFace 사의 BERT 모델 사용해서 임베딩

import re
from urllib.parse import urlparse
#외부 모델 사용하기 위한 라이브러리: transformers&torch
from transformers import BertModel, BertTokenizer
import torch
import numpy as np
import pickle


def model_process_example(url):
    return_types = {
        0:'benign',
        1:'defacement',
        2:'phishing',
        3:'malware'}
    #불필요한 수식어 제거
    url = re.sub('www.', '', url)
    url = re.sub('http://', '', url)
    url = re.sub('https://', '', url)
    url = re.sub('.html', '', url)
    url = re.sub('.htm', '', url)

    #primary url 구하기
    reg = re.compile('^(.*?)\/')
    temp = reg.findall(url)
    primary_url = '0'
    if temp:
        primary_url = temp
    else:
        primary_url = url
    #character lengths
    num_letters = sum(char.isalpha() for char in url)
    num_digits = sum(char.isdigit() for char in url)
    special_chars = "!@#$%^&*()_+-=[]{};:,.<>/?`~|"
    num_special_chars = sum(char in special_chars for char in url)
    #shortening service
    def has_shortening_ser(url):
        pattern = re.compile(r'bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                            r'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                            r'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                            r'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                            r'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                            r'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                            r'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                            r'tr\.im|link\.zip\.net')
        match = pattern.search(url)
        return int(bool(match))
    shortening_service = has_shortening_ser(url)
    #urls where hostname = domain
    def abnormal(url):
        parsed_url = urlparse(url)
        hostname = parsed_url.hostname
        if hostname:
            hostname = str(hostname)
            match = re.search(hostname, url)
            if match:
                return 1
        return 0
    abnormal_url = abnormal(url)
    #urls with IP
    def have_ip(url):
        pattern = r'(([01]?\d\d?|2[0-4]\d|25[0-5])\.([01]?\d\d?|2[0-4]\d|25[0-5])\.([01]?\d\d?|2[0-4]\d|25[0-5])\.' \
                r'([01]?\d\d?|2[0-4]\d|25[0-5])\/)|' \
                r'(([01]?\d\d?|2[0-4]\d|25[0-5])\.([01]?\d\d?|2[0-4]\d|25[0-5])\.([01]?\d\d?|2[0-4]\d|25[0-5])\.' \
                r'([01]?\d\d?|2[0-4]\d|25[0-5])\/)|' \
                r'((0x[0-9a-fA-F]{1,2})\.(0x[0-9a-fA-F]{1,2})\.(0x[0-9a-fA-F]{1,2})\.(0x[0-9a-fA-F]{1,2})\/)' \
                r'(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}|' \
                r'([0-9]+(?:\.[0-9]+){3}:[0-9]+)|' \
                r'((?:(?:\d|[01]?\d\d|2[0-4]\d|25[0-5])\.){3}(?:25[0-5]|2[0-4]\d|[01]?\d\d|\d)(?:\/\d{1,2})?)'

        match = re.search(pattern, url)
        if match:
            return 1
        else:
            return 0
    is_ip = have_ip(url)

    nonbert_features = np.array([len(url), num_letters, num_digits, num_special_chars, shortening_service, abnormal_url, is_ip]).reshape(1,-1)

    # Load the pre-trained BERT model and tokenizer
    # GPU가 없을 것이기 때문에 CPU사용
    model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    # Define a function to extract features for each transaction
    def extract_features(text):
        # Tokenize the text
        input_ids = tokenizer(text, return_tensors="pt", add_special_tokens=True)['input_ids']
        # Get the hidden states for each token
        with torch.no_grad():
            outputs = model(input_ids)
            hidden_states = outputs[2]
        # Concatenate the last 4 hidden states
        token_vecs = []
        for layer in range(-4, 0):
            token_vecs.append(hidden_states[layer][0])
        # Calculate the mean of the last 4 hidden states
        features = []
        for token in token_vecs:
            features.append(torch.mean(token, dim=0))
        # Return the features as a tensor
        return torch.stack(features)
    features = extract_features(url).numpy().reshape(1,-1)
    dataset = np.hstack((features, nonbert_features))
    #path = ' '# 모델 피클이 있을 주소
    model_bert = pickle.load(open(path+'model_bert_logreg.pkl', 'rb'))
    ans = model_bert.predict(dataset)
    print(ans)
    return return_types[ans[0]]












In [33]:
model_process_example('https://www.microsoft.com/ko-kr/windows/?r=1')

[0.]


(array([0.]), 'benign')

# ML

In [ ]:
url_data = oridf.copy()

In [ ]:
url_data['url'] = url_data['url'].replace('www.', '', regex=True)
url_data['url'] = url_data['url'].replace('http://', '', regex=True)
url_data['url'] = url_data['url'].replace('https://', '', regex=True)
url_data['url'] = url_data['url'].replace('.html', '', regex=True)
url_data['url'] = url_data['url'].replace('.htm', '', regex=True)

url_data.head()

,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko,benign
2,bopsecrets.org/rexroth/cr/1,benign
3,garage-pirenne.be/index.php?option=com_content...,defacement
4,adventure-nicaragua.net/index.php?option=com_m...,defacement


In [ ]:
url_data["url_type"] = url_data["type"].replace({
    'benign':0,
    'defacement':1,
    'phishing':2,
    'malware':3
})
url_data["bw"] = url_data["type"].replace({
    'benign':0,
    'defacement':1,
    'phishing':1,
    'malware':1
})

In [ ]:
import re
reg = re.compile('^(.*?)\/')
def pri_domain(text):
  temp = reg.findall(text)
  if temp:
    return temp[0]
  else:
    return text
url_data['pri_domain'] = url_data['url'].apply(pri_domain)

In [ ]:
url_data

,url,type,url_type,bw,pri_domain
0,br-icloud.com.br,phishing,2,1,br-icloud.com.br
1,mp3raid.com/music/krizz_kaliko,benign,0,0,mp3raid.com
2,bopsecrets.org/rexroth/cr/1,benign,0,0,bopsecrets.org
3,garage-pirenne.be/index.php?option=com_content...,defacement,1,1,garage-pirenne.be
4,adventure-nicaragua.net/index.php?option=com_m...,defacement,1,1,adventure-nicaragua.net
...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402,phishing,2,1,xbox360.ign.com
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,2,1,games.teamxbox.com
651188,gamespot.com/xbox360/action/deadspace/,phishing,2,1,gamespot.com
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,2,1,en.wikipedia.org


In [ ]:
def count_letters(url):
    num_letters = sum(char.isalpha() for char in url)
    return num_letters

def count_digits(url):
    num_digits = sum(char.isdigit() for char in url)
    return num_digits
def count_special_chars(url):
    special_chars = "!@#$%^&*()_+-=[]{};:,.<>/?`~|"
    num_special_chars = sum(char in special_chars for char in url)
    return num_special_chars

In [ ]:
url_data['len'] = oridf['url'].apply(len)

In [ ]:

url_data['letters_count'] =url_data['url'].apply(lambda x: count_letters(x))
url_data['digits_count']  =url_data['url'].apply(lambda x: count_digits(x))
url_data['special_chars_count']  =url_data['url'].apply(lambda x: count_special_chars(x))

In [ ]:
def has_shortening_service(url):
    pattern = re.compile(r'bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                         r'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                         r'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                         r'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                         r'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                         r'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                         r'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                         r'tr\.im|link\.zip\.net')
    match = pattern.search(url)
    return int(bool(match))

In [ ]:
url_data['shortened']  =url_data['url'].apply(lambda x: has_shortening_service(x))

In [ ]:
from urllib.parse import urlparse
def abnormal_url(url):
    parsed_url = urlparse(url)
    hostname = parsed_url.hostname
    if hostname:
        hostname = str(hostname)
        match = re.search(hostname, url)
        if match:
            return 1
    return 0

In [ ]:
url_data['abnormal_url']  =oridf['url'].apply(lambda x: abnormal_url(x))

In [ ]:
url_data['abnormal_url'].sum()

187990

In [ ]:
urlparse(oridf['url'][555183]).hostname

'219.155.142.211'

In [ ]:
url_data.loc[url_data['abnormal_url']==1]

,url,type,url_type,bw,pri_domain,letters_count,digits_count,special_chars_count,shortened,abnormal_url,have_ip
3,garage-pirenne.be/index.php?option=com_content...,defacement,1,1,garage-pirenne.be,56,7,14,0,1,0
4,adventure-nicaragua.net/index.php?option=com_m...,defacement,1,1,adventure-nicaragua.net,195,22,11,0,1,0
5,buzzfil.net/m/show-art/ils-etaient-loin-de-s-i...,benign,0,0,buzzfil.net,85,1,20,0,1,0
8,pashminaonline.com/pure-pashminas,defacement,1,1,pashminaonline.com,30,0,3,0,1,0
11,ikenmijnkunst.nl/index.php/exposities/expositi...,defacement,1,1,ikenmijnkunst.nl,43,4,6,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
555181,211.137.225.95:47475/Mozi.m,malware,3,1,211.137.225.95:47475,5,16,6,0,1,1
555182,115.55.246.246:44661/Mozi.m,malware,3,1,115.55.246.246:44661,5,16,6,0,1,1
555183,219.155.142.211:55621/Mozi.m,malware,3,1,219.155.142.211:55621,5,17,6,0,1,1
555184,42.238.164.8:48848/Mozi.m,malware,3,1,42.238.164.8:48848,5,14,6,0,1,1


In [ ]:
def have_ip_address(url):
    pattern = r'(([01]?\d\d?|2[0-4]\d|25[0-5])\.([01]?\d\d?|2[0-4]\d|25[0-5])\.([01]?\d\d?|2[0-4]\d|25[0-5])\.' \
              r'([01]?\d\d?|2[0-4]\d|25[0-5])\/)|' \
              r'(([01]?\d\d?|2[0-4]\d|25[0-5])\.([01]?\d\d?|2[0-4]\d|25[0-5])\.([01]?\d\d?|2[0-4]\d|25[0-5])\.' \
              r'([01]?\d\d?|2[0-4]\d|25[0-5])\/)|' \
              r'((0x[0-9a-fA-F]{1,2})\.(0x[0-9a-fA-F]{1,2})\.(0x[0-9a-fA-F]{1,2})\.(0x[0-9a-fA-F]{1,2})\/)' \
              r'(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}|' \
              r'([0-9]+(?:\.[0-9]+){3}:[0-9]+)|' \
              r'((?:(?:\d|[01]?\d\d|2[0-4]\d|25[0-5])\.){3}(?:25[0-5]|2[0-4]\d|[01]?\d\d|\d)(?:\/\d{1,2})?)'

    match = re.search(pattern, url)
    if match:
        return 1
    else:
        return 0

In [ ]:
url_data['have_ip']  =url_data['url'].apply(lambda x: have_ip_address(x))

In [ ]:
url_data['have_ip'].sum()

12488

In [ ]:
url_data.isnull

url                    0
type                   0
url_type               0
bw                     0
pri_domain             0
letters_count          0
digits_count           0
special_chars_count    0
shortened              0
abnormal_url           0
have_ip                0
dtype: int64

In [ ]:
def get_url_region(primary_domain):
    ccTLD_to_region = {
    ".ac": "Ascension Island",
    ".ad": "Andorra",
    ".ae": "United Arab Emirates",
    ".af": "Afghanistan",
    ".ag": "Antigua and Barbuda",
    ".ai": "Anguilla",
    ".al": "Albania",
    ".am": "Armenia",
    ".an": "Netherlands Antilles",
    ".ao": "Angola",
    ".aq": "Antarctica",
    ".ar": "Argentina",
    ".as": "American Samoa",
    ".at": "Austria",
    ".au": "Australia",
    ".aw": "Aruba",
    ".ax": "Åland Islands",
    ".az": "Azerbaijan",
    ".ba": "Bosnia and Herzegovina",
    ".bb": "Barbados",
    ".bd": "Bangladesh",
    ".be": "Belgium",
    ".bf": "Burkina Faso",
    ".bg": "Bulgaria",
    ".bh": "Bahrain",
    ".bi": "Burundi",
    ".bj": "Benin",
    ".bm": "Bermuda",
    ".bn": "Brunei Darussalam",
    ".bo": "Bolivia",
    ".br": "Brazil",
    ".bs": "Bahamas",
    ".bt": "Bhutan",
    ".bv": "Bouvet Island",
    ".bw": "Botswana",
    ".by": "Belarus",
    ".bz": "Belize",
    ".ca": "Canada",
    ".cc": "Cocos Islands",
    ".cd": "Democratic Republic of the Congo",
    ".cf": "Central African Republic",
    ".cg": "Republic of the Congo",
    ".ch": "Switzerland",
    ".ci": "Côte d'Ivoire",
    ".ck": "Cook Islands",
    ".cl": "Chile",
    ".cm": "Cameroon",
    ".cn": "China",
    ".co": "Colombia",
    ".cr": "Costa Rica",
    ".cu": "Cuba",
    ".cv": "Cape Verde",
    ".cw": "Curaçao",
    ".cx": "Christmas Island",
    ".cy": "Cyprus",
    ".cz": "Czech Republic",
    ".de": "Germany",
    ".dj": "Djibouti",
    ".dk": "Denmark",
    ".dm": "Dominica",
    ".do": "Dominican Republic",
    ".dz": "Algeria",
    ".ec": "Ecuador",
    ".ee": "Estonia",
    ".eg": "Egypt",
    ".er": "Eritrea",
    ".es": "Spain",
    ".et": "Ethiopia",
    ".eu": "European Union",
    ".fi": "Finland",
    ".fj": "Fiji",
    ".fk": "Falkland Islands",
    ".fm": "Federated States of Micronesia",
    ".fo": "Faroe Islands",
    ".fr": "France",
    ".ga": "Gabon",
    ".gb": "United Kingdom",
    ".gd": "Grenada",
    ".ge": "Georgia",
    ".gf": "French Guiana",
    ".gg": "Guernsey",
    ".gh": "Ghana",
    ".gi": "Gibraltar",
    ".gl": "Greenland",
    ".gm": "Gambia",
    ".gn": "Guinea",
    ".gp": "Guadeloupe",
    ".gq": "Equatorial Guinea",
    ".gr": "Greece",
    ".gs": "South Georgia and the South Sandwich Islands",
    ".gt": "Guatemala",
    ".gu": "Guam",
    ".gw": "Guinea-Bissau",
    ".gy": "Guyana",
    ".hk": "Hong Kong",
    ".hm": "Heard Island and McDonald Islands",
    ".hn": "Honduras",
    ".hr": "Croatia",
    ".ht": "Haiti",
    ".hu": "Hungary",
    ".id": "Indonesia",
    ".ie": "Ireland",
    ".il": "Israel",
    ".im": "Isle of Man",
    ".in": "India",
    ".io": "British Indian Ocean Territory",
    ".iq": "Iraq",
    ".ir": "Iran",
    ".is": "Iceland",
    ".it": "Italy",
    ".je": "Jersey",
    ".jm": "Jamaica",
    ".jo": "Jordan",
    ".jp": "Japan",
    ".ke": "Kenya",
    ".kg": "Kyrgyzstan",
    ".kh": "Cambodia",
    ".ki": "Kiribati",
    ".km": "Comoros",
    ".kn": "Saint Kitts and Nevis",
    ".kp": "Democratic People's Republic of Korea (North Korea)",
    ".kr": "Republic of Korea (South Korea)",
    ".kw": "Kuwait",
    ".ky": "Cayman Islands",
    ".kz": "Kazakhstan",
    ".la": "Laos",
    ".lb": "Lebanon",
    ".lc": "Saint Lucia",
    ".li": "Liechtenstein",
    ".lk": "Sri Lanka",
    ".lr": "Liberia",
    ".ls": "Lesotho",
    ".lt": "Lithuania",
    ".lu": "Luxembourg",
    ".lv": "Latvia",
    ".ly": "Libya",
    ".ma": "Morocco",
    ".mc": "Monaco",
    ".md": "Moldova",
    ".me": "Montenegro",
    ".mf": "Saint Martin (French part)",
    ".mg": "Madagascar",
    ".mh": "Marshall Islands",
    ".mk": "North Macedonia",
    ".ml": "Mali",
    ".mm": "Myanmar",
    ".mn": "Mongolia",
    ".mo": "Macao",
    ".mp": "Northern Mariana Islands",
    ".mq": "Martinique",
    ".mr": "Mauritania",
    ".ms": "Montserrat",
    ".mt": "Malta",
    ".mu": "Mauritius",
    ".mv": "Maldives",
    ".mw": "Malawi",
    ".mx": "Mexico",
    ".my": "Malaysia",
    ".mz": "Mozambique",
    ".na": "Namibia",
    ".nc": "New Caledonia",
    ".ne": "Niger",
    ".nf": "Norfolk Island",
    ".ng": "Nigeria",
    ".ni": "Nicaragua",
    ".nl": "Netherlands",
    ".no": "Norway",
    ".np": "Nepal",
    ".nr": "Nauru",
    ".nu": "Niue",
    ".nz": "New Zealand",
    ".om": "Oman",
    ".pa": "Panama",
    ".pe": "Peru",
    ".pf": "French Polynesia",
    ".pg": "Papua New Guinea",
    ".ph": "Philippines",
    ".pk": "Pakistan",
    ".pl": "Poland",
    ".pm": "Saint Pierre and Miquelon",
    ".pn": "Pitcairn",
    ".pr": "Puerto Rico",
    ".ps": "Palestinian Territory",
    ".pt": "Portugal",
    ".pw": "Palau",
    ".py": "Paraguay",
    ".qa": "Qatar",
    ".re": "Réunion",
    ".ro": "Romania",
    ".rs": "Serbia",
    ".ru": "Russia",
    ".rw": "Rwanda",
    ".sa": "Saudi Arabia",
    ".sb": "Solomon Islands",
    ".sc": "Seychelles",
    ".sd": "Sudan",
    ".se": "Sweden",
    ".sg": "Singapore",
    ".sh": "Saint Helena",
    ".si": "Slovenia",
    ".sj": "Svalbard and Jan Mayen",
    ".sk": "Slovakia",
    ".sl": "Sierra Leone",
    ".sm": "San Marino",
    ".sn": "Senegal",
    ".so": "Somalia",
    ".sr": "Suriname",
    ".ss": "South Sudan",
    ".st": "São Tomé and Príncipe",
    ".sv": "El Salvador",
    ".sx": "Sint Maarten (Dutch part)",
    ".sy": "Syria",
    ".sz": "Eswatini",
    ".tc": "Turks and Caicos Islands",
    ".td": "Chad",
    ".tf": "French Southern Territories",
    ".tg": "Togo",
    ".th": "Thailand",
    ".tj": "Tajikistan",
    ".tk": "Tokelau",
    ".tl": "Timor-Leste",
    ".tm": "Turkmenistan",
    ".tn": "Tunisia",
    ".to": "Tonga",
    ".tr": "Turkey",
    ".tt": "Trinidad and Tobago",
    ".tv": "Tuvalu",
    ".tw": "Taiwan",
    ".tz": "Tanzania",
    ".ua": "Ukraine",
    ".ug": "Uganda",
    ".uk": "United Kingdom",
    ".us": "United States",
    ".uy": "Uruguay",
    ".uz": "Uzbekistan",
    ".va": "Vatican City",
    ".vc": "Saint Vincent and the Grenadines",
    ".ve": "Venezuela",
    ".vg": "British Virgin Islands",
    ".vi": "U.S. Virgin Islands",
    ".vn": "Vietnam",
    ".vu": "Vanuatu",
    ".wf": "Wallis and Futuna",
    ".ws": "Samoa",
    ".ye": "Yemen",
    ".yt": "Mayotte",
    ".za": "South Africa",
    ".zm": "Zambia",
    ".zw": "Zimbabwe"
    }

    for ccTLD in ccTLD_to_region:
        if primary_domain.endswith(ccTLD):
            return ccTLD_to_region[ccTLD]

    return "Global"

In [ ]:
url_data['url_region'] = url_data['pri_domain'].apply(lambda x: get_url_region(str(x)))

In [ ]:
!pip install tldextract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
import tldextract
def extract_root_domain(url):
    extracted = tldextract.extract(url)
    root_domain = extracted.domain
    return root_domain

In [ ]:
url_data['root_domain'] = url_data['pri_domain'].apply(lambda x: extract_root_domain(str(x)))

In [ ]:
url_data.isnull().sum()

url                    0
type                   0
url_type               0
bw                     0
pri_domain             0
letters_count          0
digits_count           0
special_chars_count    0
shortened              0
abnormal_url           0
have_ip                0
url_region             0
root_domain            0
dtype: int64

In [ ]:
url_data.duplicated().sum()

10066

In [ ]:
url_data.drop_duplicates(inplace=True)

In [ ]:
url_data

,url,type,url_type,bw,pri_domain,len,letters_count,digits_count,special_chars_count,shortened,abnormal_url,have_ip,url_region,root_domain
0,br-icloud.com.br,phishing,2,1,br-icloud.com.br,16,13,0,3,0,0,0,Brazil,br-icloud
1,mp3raid.com/music/krizz_kaliko,benign,0,0,mp3raid.com,35,25,1,4,0,0,0,Global,mp3raid
2,bopsecrets.org/rexroth/cr/1,benign,0,0,bopsecrets.org,31,22,1,4,0,0,0,Global,bopsecrets
3,garage-pirenne.be/index.php?option=com_content...,defacement,1,1,garage-pirenne.be,88,56,7,14,0,1,0,Belgium,garage-pirenne
4,adventure-nicaragua.net/index.php?option=com_m...,defacement,1,1,adventure-nicaragua.net,235,195,22,11,0,1,0,Global,adventure-nicaragua
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402,phishing,2,1,xbox360.ign.com,39,17,12,5,0,0,0,Global,ign
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,2,1,games.teamxbox.com,44,29,7,8,1,0,0,Global,teamxbox
651188,gamespot.com/xbox360/action/deadspace/,phishing,2,1,gamespot.com,42,30,3,5,1,0,0,Global,gamespot
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,2,1,en.wikipedia.org,45,36,0,9,0,0,0,Global,wikipedia


In [ ]:
url_data[url_data['root_domain'] == 'wikipedia']['type'].value_counts()

benign      12876
phishing      550
Name: type, dtype: int64

In [ ]:
url_data.to_csv("/content/MyDrive/MyDrive/네트워크 과정 프젝/preprocessed.csv")

In [ ]:
url_data = pd.read_csv(path+'preprocessed.csv')

In [ ]:
url_data

,Unnamed: 0,url,type,url_type,bw,pri_domain,len,letters_count,digits_count,special_chars_count,shortened,abnormal_url,have_ip,url_region,root_domain
0,0,br-icloud.com.br,phishing,2,1,br-icloud.com.br,16,13,0,3,0,0,0,Brazil,br-icloud
1,1,mp3raid.com/music/krizz_kaliko,benign,0,0,mp3raid.com,35,25,1,4,0,0,0,Global,mp3raid
2,2,bopsecrets.org/rexroth/cr/1,benign,0,0,bopsecrets.org,31,22,1,4,0,0,0,Global,bopsecrets
3,3,garage-pirenne.be/index.php?option=com_content...,defacement,1,1,garage-pirenne.be,88,56,7,14,0,1,0,Belgium,garage-pirenne
4,4,adventure-nicaragua.net/index.php?option=com_m...,defacement,1,1,adventure-nicaragua.net,235,195,22,11,0,1,0,Global,adventure-nicaragua
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641120,651186,xbox360.ign.com/objects/850/850402,phishing,2,1,xbox360.ign.com,39,17,12,5,0,0,0,Global,ign
641121,651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,2,1,games.teamxbox.com,44,29,7,8,1,0,0,Global,teamxbox
641122,651188,gamespot.com/xbox360/action/deadspace/,phishing,2,1,gamespot.com,42,30,3,5,1,0,0,Global,gamespot
641123,651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,2,1,en.wikipedia.org,45,36,0,9,0,0,0,Global,wikipedia


In [ ]:
import hashlib
def hash_encode(category):
    hash_object = hashlib.md5()
    hash_object.update(category.encode(encoding='UTF-8'))
    return int(hash_object.hexdigest(), 16) % (10 ** 8)

In [ ]:
# Apply hash encoding to the categorical feature
url_data['root_domain'] = url_data['root_domain'].astype(str)
url_data['url_region'] = url_data['url_region'].astype(str)

url_data['root_domain'] = url_data['root_domain'].apply(hash_encode)
url_data['url_region'] = url_data['url_region'].apply(hash_encode)

In [ ]:
url_data.columns

Index(['Unnamed: 0', 'url', 'type', 'url_type', 'bw', 'pri_domain', 'len',
       'letters_count', 'digits_count', 'special_chars_count', 'shortened',
       'abnormal_url', 'have_ip', 'url_region', 'root_domain'],
      dtype='object')

In [ ]:
dftest = url_data.drop(['Unnamed: 0', 'url', 'type','pri_domain'], axis=1)

In [ ]:
dftest[['len', 'letters_count', 'digits_count', 'special_chars_count', 'url_region', 'root_domain']]

,len,letters_count,digits_count,special_chars_count,url_region,root_domain
0,0.006900,0.006080,0.000000,0.00813,0.275105,0.013106
1,0.015639,0.011693,0.000831,0.01084,0.324004,0.583357
2,0.013799,0.010290,0.000831,0.01084,0.324004,0.286117
3,0.040018,0.026193,0.005814,0.03794,0.714764,0.890454
4,0.107636,0.091207,0.018272,0.02981,0.324004,0.768387
...,...,...,...,...,...,...
641120,0.017479,0.007951,0.009967,0.01355,0.324004,0.476324
641121,0.019779,0.013564,0.005814,0.02168,0.324004,0.338424
641122,0.018859,0.014032,0.002492,0.01355,0.324004,0.614294
641123,0.020239,0.016838,0.000000,0.02439,0.324004,0.415401


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler_ = MinMaxScaler()
scaler_.fit(dftest[['len', 'letters_count', 'digits_count', 'special_chars_count', 'url_region', 'root_domain']])
dftest[['len', 'letters_count', 'digits_count', 'special_chars_count', 'url_region', 'root_domain']] = scaler_.transform(dftest[['len', 'letters_count', 'digits_count', 'special_chars_count', 'url_region', 'root_domain']])

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
x = dftest.drop(columns=['url_type','bw'])
y = dftest['url_type']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from sklearn.pipeline import Pipeline



classifiers = [
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    KNeighborsClassifier(),
    ExtraTreesClassifier(),
    GaussianNB()
]

In [ ]:
results = []
for classifier in classifiers:
    # Create a pipeline with the scaler and classifier
    pipeline = Pipeline([
        ('classifier', classifier)
    ])
    # Perform cross-validation using the pipeline
    scores = cross_val_score(pipeline, x, y, cv=2, scoring='accuracy')
    # Calculate additional evaluation metrics
    y_pred = cross_val_predict(pipeline, x, y, cv=2)
    accuracy = accuracy_score(y, y_pred)
    recall = recall_score(y, y_pred, average='weighted')
    precision = precision_score(y, y_pred, average='weighted', zero_division=1)  # Set zero_division=1
    f1 = f1_score(y, y_pred, average='weighted')
    # Append the results to the list
    results.append((classifier.__class__.__name__, accuracy, recall, precision, f1))

In [ ]:
results = pd.DataFrame(results, columns=['Classifier', 'Accuracy', 'Recall', 'Precision', 'F1-Score'])
results = results.sort_values(by='Accuracy', ascending=False)
results

,Classifier,Accuracy,Recall,Precision,F1-Score
1,RandomForestClassifier,0.820987,0.820987,0.755030,0.775275
4,ExtraTreesClassifier,0.820987,0.820987,0.752763,0.774567
3,KNeighborsClassifier,0.815059,0.815059,0.752681,0.772790
0,DecisionTreeClassifier,0.806161,0.806161,0.749586,0.771101
2,AdaBoostClassifier,0.787201,0.787201,0.707777,0.736446
5,GaussianNB,0.533918,0.533918,0.688867,0.568626


# BERT

In [ ]:
!pip -q install transformers

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import cross_val_score

In [ ]:
url_data = pd.read_csv(path+'preprocessed.csv')

In [ ]:
url_data

,Unnamed: 0,url,type,url_type,bw,pri_domain,len,letters_count,digits_count,special_chars_count,shortened,abnormal_url,have_ip,url_region,root_domain
0,0,br-icloud.com.br,phishing,2,1,br-icloud.com.br,16,13,0,3,0,0,0,Brazil,br-icloud
1,1,mp3raid.com/music/krizz_kaliko,benign,0,0,mp3raid.com,35,25,1,4,0,0,0,Global,mp3raid
2,2,bopsecrets.org/rexroth/cr/1,benign,0,0,bopsecrets.org,31,22,1,4,0,0,0,Global,bopsecrets
3,3,garage-pirenne.be/index.php?option=com_content...,defacement,1,1,garage-pirenne.be,88,56,7,14,0,1,0,Belgium,garage-pirenne
4,4,adventure-nicaragua.net/index.php?option=com_m...,defacement,1,1,adventure-nicaragua.net,235,195,22,11,0,1,0,Global,adventure-nicaragua
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641120,651186,xbox360.ign.com/objects/850/850402,phishing,2,1,xbox360.ign.com,39,17,12,5,0,0,0,Global,ign
641121,651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,2,1,games.teamxbox.com,44,29,7,8,1,0,0,Global,teamxbox
641122,651188,gamespot.com/xbox360/action/deadspace/,phishing,2,1,gamespot.com,42,30,3,5,1,0,0,Global,gamespot
641123,651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,2,1,en.wikipedia.org,45,36,0,9,0,0,0,Global,wikipedia


In [ ]:
import hashlib
def hash_encode(category):
    hash_object = hashlib.md5()
    hash_object.update(category.encode(encoding='UTF-8'))
    return int(hash_object.hexdigest(), 16) % (10 ** 8)

In [ ]:
# Apply hash encoding to the categorical feature
url_data['root_domain'] = url_data['root_domain'].astype(str)
url_data['url_region'] = url_data['url_region'].astype(str)

url_data['root_domain'] = url_data['root_domain'].apply(hash_encode)
url_data['url_region'] = url_data['url_region'].apply(hash_encode)

In [ ]:
#from sklearn.preprocessing import StandardScaler, MinMaxScaler
#scaler_ = MinMaxScaler()
#scaler_.fit(url_data[['len', 'letters_count', 'digits_count', 'special_chars_count', 'url_region', 'root_domain']])
#url_data[['len', 'letters_count', 'digits_count', 'special_chars_count', 'url_region', 'root_domain']] = scaler_.transform(url_data[['len', 'letters_count', 'digits_count', 'special_chars_count', 'url_region', 'root_domain']])

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

# Load the pre-trained BERT model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
sentence  = 'Hello World!'

inputs    = tokenizer(sentence, return_tensors="pt",add_special_tokens=True).to(device)
model     = model.to(device)
outputs   = model(**inputs)

In [ ]:
# Define a function to extract features for each transaction
def extract_features(text):
    # Tokenize the text
    input_ids = tokenizer(text, return_tensors="pt", add_special_tokens=True).to(device)['input_ids']
    # Get the hidden states for each token
    with torch.no_grad():
        outputs = model(input_ids)
        hidden_states = outputs[2]
    # Concatenate the last 4 hidden states
    token_vecs = []
    for layer in range(-4, 0):
        token_vecs.append(hidden_states[layer][0])
    # Calculate the mean of the last 4 hidden states
    features = []
    for token in token_vecs:
        features.append(torch.mean(token, dim=0))
    # Return the features as a tensor
    return torch.stack(features)

In [ ]:
dftest = url_data.drop(['Unnamed: 0',  'type'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

x = dftest.drop(columns=['url_type','bw'])
y = dftest['url_type']
_,x_data,__,y_data = train_test_split(x,y,test_size=0.15, random_state=52)

In [ ]:
x_data.size

1057859

In [ ]:
tokenizer(x_data.iloc[0]["url"], return_tensors="pt", add_special_tokens=True).to(device)['input_ids'].shape

torch.Size([1, 21])

In [ ]:
tokenizer(x_data.iloc[0]["url"], return_tensors="pt", add_special_tokens=True)['input_ids'].shape

torch.Size([1, 21])

In [ ]:
# Extract features for each transaction
features = []
for i in range(len(x_data)):
    if len(x_data.iloc[i]["url"]) < 512:
      features.append(extract_features(x_data.iloc[i]["url"]))
    else:
      features.append(extract_features(x_data.iloc[i]["pri_domain"][:511]))
    if i%10000==0:
      print(i)
# Concatenate the features and convert to a numpy array
features = torch.cat(features).cpu().numpy()

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [ ]:
len(features)

384676

In [ ]:
len(x_data)

96169

In [ ]:
# features is a 2D numpy array of size 4000x768
# labels is a 1D numpy array of size 1000
# reshape the feature array to size 1000x(768*4)

features_reshaped = features.reshape((len(x_data), -1))

# concatenate the feature array with the label array horizontally
#dataset = np.hstack((features_reshaped, y_data.values.reshape(-1, 1)))

# dataset is a 2D numpy array of size 1000x(4000*768+1)

In [ ]:
features_reshaped.nbytes

1181724672

In [ ]:
features_reshaped.shape

(160282, 3072)

In [ ]:
temp = x_data[['len','letters_count','digits_count', 'special_chars_count', 'shortened', 'abnormal_url', 'have_ip']].values.reshape(-1, 7)
#temp = np.hstack((temp, y_data.values.reshape(-1,1)))
temp.shape

(96169, 7)

In [ ]:
dataset = np.hstack((features_reshaped, temp))
dataset = np.hstack((dataset, y_data.values.reshape(-1,1)))

In [ ]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Convert the training and testing sets back into separate feature and label arrays
X_train, y_train = train_data[:, :-1], train_data[:, -1]
X_test, y_test = test_data[:, :-1], test_data[:, -1]

In [ ]:
from imblearn.over_sampling import SMOTE

# Instantiate SMOTE
sm = SMOTE(random_state=42)

# Fit SMOTE to training data
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

y_train_s = pd.Series(y_train)
y_train_res = pd.Series (y_train_res)

# Print class distribution of original and resampled data
print('Class distribution before resampling:', y_train_s.value_counts())
print('Class distribution after resampling:', y_train_res.value_counts())

Class distribution before resampling: 0.0    17047
1.0     3835
2.0     3770
3.0      993
dtype: int64
Class distribution after resampling: 0.0    17047
3.0    17047
1.0    17047
2.0    17047
dtype: int64


In [ ]:
from sklearn.linear_model import LogisticRegression

# Train a logistic regression classifier on the training set
clf = LogisticRegression(max_iter=500)
clf.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=500)

In [ ]:
from sklearn import svm


# Train a svc classifier on the training set
# lags on colab
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)

In [ ]:
# Evaluate the classifier on the testing set
score = clf.score(X_test, y_test)
score

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
y_pred = clf.predict(X_test)
# Generate the confusion matrix and classification report
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

In [ ]:
print("Confusion Matrix:\n", cm)

In [ ]:
print("\nClassification Report:\n", cr)

In [ ]:
import pickle

pickle.dump(clf, open(path+'model_bert_svc.pkl', 'wb'))

In [ ]:
model_bert = pickle.load(open(path+'model_bert_logreg.pkl', 'rb'))
model_bert.score(X_test, y_test)

0.9453051887282936